<a href="https://colab.research.google.com/github/usmangul23/UserManualChatbotV1/blob/main/PDF_Manual_V3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentence-transformers groq vector-db-client
!pip install qdrant-client # Install the qdrant-client library
!pip install faiss-cpu
!pip install google-generativeai
!pip install python-dotenv
!pip install langchain
!pip install langchain_google_genai
!pip install langchain-community
!pip install streamlit
!pip install pypdf2
!pip install pyngrok


ERROR: Could not find a version that satisfies the requirement vector-db-client (from versions: none)
ERROR: No matching distribution found for vector-db-client
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.2/267.2 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.9/5.9 MB 69.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.6/316.6 kB 19.3 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
  Attempting uninstall: grpcio
    Found existing installation: grpcio 1.64.1
    Uninstalling grpcio-1.64.1:
      Successfully uninstalled grpcio-1.64.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the

In [ ]:
import io
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
import google.generativeai as genai
from google.colab import files, userdata

# Retrieve the Google API key securely
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

# Step 1: Extract and decrypt PDF text, track page numbers
def get_pdf_text(pdf_docs):
    pages_text = []
    for pdf_data in pdf_docs:
        pdf_file = io.BytesIO(pdf_data)  # Convert bytes to file-like object
        pdf_reader = PdfReader(pdf_file)
        if pdf_reader.is_encrypted:
            try:
                pdf_reader.decrypt("")
            except Exception as e:
                print(f"Failed to decrypt PDF: {e}")
                return []
        for page_num, page in enumerate(pdf_reader.pages, start=1):
            page_text = page.extract_text()
            if page_text:
                pages_text.append({"text": page_text, "page_num": page_num})
    return pages_text

# Step 2: Split text into chunks and keep track of page numbers
def get_text_chunks(pages_text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    chunks_with_pages = []
    for page in pages_text:
        page_chunks = text_splitter.split_text(page["text"])
        for chunk in page_chunks:
            chunks_with_pages.append({"text": chunk, "page_num": page["page_num"]})
    return chunks_with_pages

# Step 3: Create or load a vector store
def load_or_create_vector_store(text_chunks):
    texts = [chunk["text"] for chunk in text_chunks]
    embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001", google_api_key=GOOGLE_API_KEY)
    vector_store = FAISS.from_texts(texts, embedding=embeddings)
    return vector_store

# Step 4: Create the conversational chain
def get_conversational_chain():
    prompt_template = """
    Provide a detailed answer to the question based on the context provided. Elaborate as much as possible and include any relevant details.

    Context:\n{context}\n
    Question:\n{question}\n
    Detailed Answer:
    """
    model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0.3, google_api_key=GOOGLE_API_KEY)
    prompt = PromptTemplate(template=prompt_template, input_variables=["context", "question"])
    chain = load_qa_chain(model, chain_type="stuff", prompt=prompt)
    return chain

# Step 5: User input and generate detailed response with page numbers
def user_input(user_question, chain, vector_store, text_chunks):
    docs = vector_store.similarity_search(user_question)  # Retrieve relevant chunks
    matching_chunks = []

    # Identify page numbers and create a list of matching chunks
    for doc in docs:
        chunk_index = [i for i, chunk in enumerate(text_chunks) if chunk["text"] == doc.page_content]
        if chunk_index:
            matching_chunks.append({
                "text": doc.page_content,
                "page_num": text_chunks[chunk_index[0]]["page_num"]
            })

    # Use the conversational chain to generate a detailed answer based on context
    response = chain({"input_documents": docs, "question": user_question}, return_only_outputs=True)
    answer_text = response['output_text']

    # Check if the response is relevant by assessing matching chunks and response length
    if not matching_chunks or len(answer_text.strip()) < 20:  # Adjust this length as needed
        return "The question does not seem to relate to the content of the PDF. Please try asking something relevant to the document."

    # Build response with page numbers if relevant
    detailed_response = answer_text + "\n\n"
    for chunk in matching_chunks:
        detailed_response += f"(Found on page {chunk['page_num']})\n\n{chunk['text']}\n\n"

    return detailed_response

# Main function for loading PDF, processing it, and answering questions
def main():
    # Upload PDF file in Colab
    uploaded_file = files.upload()
    pdf_docs = list(uploaded_file.values())  # Extract byte contents of each PDF

    # Process PDF text
    pages_text = get_pdf_text(pdf_docs)
    if not pages_text:
        print("No text extracted from the PDF. Please check the content.")
        return

    # Split text into chunks with page numbers
    text_chunks = get_text_chunks(pages_text)

    # Load or create vector store
    vector_store = load_or_create_vector_store(text_chunks)

    # Initialize conversational chain
    chain = get_conversational_chain()

    # Input loop for user questions
    print("You can now start asking questions about the PDF content:")
    while True:
        user_question = input("Enter your question (or type 'exit' to stop): ")
        if user_question.lower() == 'exit':
            break
        elif not user_question:
            print("Please enter a question.")
            continue
        else:
            # Generate and display a detailed response with page numbers
            answer = user_input(user_question, chain, vector_store, text_chunks)
            print(f"Detailed Answer: {answer}\n")

# Run the main function
if __name__ == "__main__":
    main()
